-----

# **BrainStation Bootcamp:**
### **Book Recommendation System - Data Processing**

* Author: Rurick Alejandro Granados Figueredo
* Contact: rurickgrfi@gmail.com
* Date: July 31, 2023
-----------

## **Project Overview**
Discovering the perfect book that aligns with our personal preferences can be challenging, especially for those who are not regular readers. In my capstone project, I have chosen to address this issue by building a book recommendation system. By developing this tool, I aim to predict and suggest books that individuals would enjoy reading, making the process of book selection easier and more personalized.




## **Dataset**

For this project, we utilized the Amazon Books Reviews dataset, which comprises 2 files. The first file consists of Book Reviews, gathering feedback from 3 million users on 212,404 distinct books. It forms part of the Amazon review Dataset and encompasses product reviews and metadata from Amazon, containing an impressive 142.8 million reviews covering the period from May 1996 to July 2014.

The second file, known as the Books Details file, contains comprehensive information about the same 212,404 unique books. This data was obtained by utilizing the Google Books API, ensuring that each book's details and ratings were thoroughly documented. 


### **Data Dictionary**
---

### Books Details File
|Features|Description|
|:------------|  ---------------:|
|Title|Book Title|
|description|Description of the book|
|authors|Name of book authors|
|image|url for book cover|
|previewLink|link to access this book on google Books|
|publisher|Name of the publisheer|
|publishedDate|the date of publish|
|infoLink|link to get more information about the book on google books|
|categories|genres of books|
|ratingsCount|averaging rating for book|


### Reviews File
|Features|Description|
|:------------|  ---------------:|
|id|The Id of Book|
|Title|Book Title|
|Price|The price of Book|
|User_id|Id of the user who rates the book|
|profileName|Name of the user who rates the book|
|review/helpfulness|helpfulness rating of the review, e.g. 2/3|
|review/score|rating from 0 to 5 for the book|
|review/time|time of given the review|
|review/summary|the summary of a text review|
|review/text|the full text of a review|

---


## **Table of Contents** 
---
- [1. Loading Data and Libraries Setup](#_1)  
- [2. Data Preparation and Exploration](#_2)
    - [2.1 `ratings` data exploration](#_2.1)    
        - [2.1.1 Cleaning `ratings`](#_2.1.1) 
        - [2.1.2 Feature Engineering](#_2.1.2) 
    - [2.2 `books` data exploration](#_2.2) 
        - [2.2.1 Cleaning `books`](#_2.2.1) 
        - [2.2.2 Feature Engineering](#_2.2.2) 
- [3. Merging `books` and `ratings` data frames](#_3)
    - [3.1 Data Cleaning](#_3.1)  
    - [3.2 Feature Enginering](#_3.2)  
- [4. Save Processed Data](#_4) 

-----

## 1. Loading Data and Libraries Setup <a class="anchor" id="_1"></a>

In [1]:
# Import libraries needed for the project
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Read the Books_rating csv file and asigne it to a data frame ratings
ratings = pd.read_csv('Books_rating.csv')

In [3]:
# Read the Books_data csv file and asigne it to a data frame books
books = pd.read_csv('books_data.csv')

## 2. Data Preparation and Exploration <a class="anchor" id="_2"></a>

### 2.1 `ratings` data exploration <a class="anchor" id="_2.1"></a>

In [4]:
ratings.sample(1)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
1519757,1580624073,"Make Up, Don't Break Up: Finding and Keeping L...",NaN,A1HGNXVD4859MZ,Freda J. Hill,22/23,5.0,1004486400,This book is the best kept secret for troubled...,"If you think, &quot;Men are from Mars, Women a..."


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


In [6]:
# ratings shape
print(f"Total rows: {ratings.shape[0]}")
print(f"Total columns: {ratings.shape[1]}")
print(f"Null values: {ratings.isna().sum().sum()}")

Total rows: 3000000
Total columns: 10
Null values: 3643144


In [7]:
# ratings NaN values
ratings.isna().sum()

Id                          0
Title                     208
Price                 2518829
User_id                561787
profileName            561905
review/helpfulness          0
review/score                0
review/time                 0
review/summary            407
review/text                 8
dtype: int64

#### 2.1.1 Cleaning `ratings` data frame: <a class="anchor" id="_2.1.1"></a>

##### A. Dropping NaN values (Title, User_id)
- The `Title` and `User_id` columns are essential for the project, and therefore, any rows with NaN values in these columns will be removed.

In [8]:
ratings.dropna(subset=["Title","User_id"],inplace = True)

In [9]:
ratings.isna().sum()

Id                          0
Title                       0
Price                 2023443
User_id                     0
profileName               118
review/helpfulness          0
review/score                0
review/time                 0
review/summary            397
review/text                 1
dtype: int64

The `review/text` and  `review/summary` columns have 1 and 397 NaN values each, since this amounts are not significant, those records will be droped.

In [10]:
# drop review/summary NaN values
ratings.dropna(subset=["review/summary"],inplace = True)

# drop review/text NaN values
ratings.dropna(subset=["review/text"],inplace = True)

In [11]:
ratings.shape

(2437620, 10)

##### B. Duplicate rows

In [12]:
# Calculate the number of duplicated rows
ratings.duplicated().sum()

5546

In [13]:
# Drop the duplicated rows
ratings.drop_duplicates(inplace=True)
ratings.shape

(2432074, 10)

##### C. Cleanign Book Title column

Cleaning title column from ratings data frame since this column will be used for to merge the books dataset


In [14]:
# Remove the extra space from the title column
ratings['Title'] = ratings['Title'].str.strip()

# remove the punctuation from the Title column
ratings['Title'] = ratings['Title'].str.replace('[^\w\s]','')

In [15]:
clean_title = ratings['Title'].value_counts()
clean_title.head(5)

Title
The Hobbit             17854
Pride and Prejudice    17739
Atlas Shrugged         10741
Wuthering Heights       8570
Great Expectations      5648
Name: count, dtype: int64

Identify the number of unique users and titles:

In [16]:
print("Shape:", ratings.shape)
print("Number of Unique Users:",len(ratings["User_id"].unique()))
print("Number of Unique Titles:",len(ratings["Title"].unique()))

Shape: (2432074, 10)
Number of Unique Users: 1008822
Number of Unique Titles: 206706


#### 2.1.2 `ratings` feature engineering: <a class="anchor" id="_2.1.2"></a>

In [17]:
ratings.sample(1)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
2590982,B0000640E8,The Buffalo Soldier,NaN,A1YC6AYVXK7NRQ,"DesDaz ""torr0015""",0/0,5.0,1291852800,A wonderful story,"I really, really enjoyed this book. I found th..."


##### A. New column: Review_Counts

In [18]:
# Create a new variable with the number records per each title
title_counts = ratings['Title'].value_counts()
title_counts.head(5)

Title
The Hobbit             17854
Pride and Prejudice    17739
Atlas Shrugged         10741
Wuthering Heights       8570
Great Expectations      5648
Name: count, dtype: int64

The number of records per title is equal to the number of reviews per title, 
Therefore we can create a new column to the ratings data frame with the number of reviews per title `Review_Counts`

In [19]:
ratings["Review_Counts"] = ratings["Title"].map(title_counts)
ratings.sample(3)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,Review_Counts
748494,1560975393,Palomar: The Heartbreak Soup Stories (Love and...,NaN,A3GJGBF7FO8WDJ,"Tom Knapp ""Rambles.NET editor""",11/11,5.0,1075248000,"A living, breathing town...",Palomar is just shy of being an offbeat spot o...,14
2057522,0321334876,Effective C++: 55 Specific Ways to Improve You...,NaN,A39C15CHMOE37U,"Lykos ""Lykos""",1/1,5.0,1228348800,Yet Another Must Own C++ Title!,This Book should be on every C++ programmers s...,67
604822,B000KAIZX4,"Lion, the Witch, and the Wardrobe",NaN,A7FJVRLMG5BF9,Michael JR Jose,2/2,5.0,1154908800,"THE WORLD OF NARNIA, SO LIKE HOME","Set early in World War II, four children are e...",352


In [20]:
# Checking the new column Review_Counts
# Searching the review Id=B000NBH78O to verify the Review_Counts column
ratings[ratings["Id"] == "B000NBH78O"]

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,Review_Counts
2363198,B000NBH78O,"French Hand Sewing, Vol. 1",NaN,A1PHDSHG09FQYT,Virginia Popper,2/3,5.0,1187136000,Essential Reference,I am glad that out-of-print books are so readi...,4
2363199,B000NBH78O,"French Hand Sewing, Vol. 1",NaN,A3TVB34TUT4DOF,T.Lizzee,0/0,5.0,1355875200,By the master of French hand sewing,"Beautiful book, the only one I have found with...",4
2363200,B000NBH78O,"French Hand Sewing, Vol. 1",NaN,A6RDL9WUMSGWA,S. Roncevaux,0/0,5.0,1289433600,A Great Book!,I haven't put the instructions to actual pract...,4
2363201,B000NBH78O,"French Hand Sewing, Vol. 1",NaN,A29HWEN7ZBFENT,wild child's mom,0/0,5.0,1252713600,Fabulous!,This invaluable book covers all aspects of bea...,4


- #### B. Selecting Columns for the desired data frame
    - `Price`: Will not be consider for the analysis due to having 2023443 NaN vaules which is around 82% of the records. 
    - `profileName`: Will not be consider for the analysis since we would use `User_id` as the identifier.
    - `review/time`, `review/helpfulness` : Will not be consider for the analysis.

In [21]:
rating_df = ratings[["Id","Title","User_id","review/score","review/summary","review/text","Review_Counts"]]

rating_df = rating_df.rename(columns={
    "Id": "RatingID",
    "Title": "BookTitle",
    "User_id": "UserID",
    "review/score": "Review_Score",
    "review/summary": "Review_Summary",
    "review/text": "Review_Text",
    "Review_Counts": "Review_Counts"
})

rating_df.sample(3)

,RatingID,BookTitle,UserID,Review_Score,Review_Summary,Review_Text,Review_Counts
1193310,B000CEXDTO,Midnight in the Garden of Good and Evil,AQIG9HZFIDDRV,4.0,Makes me want to go to Savannah,I enjoyed this book because it stands out from...,416
1513130,B0009RJTTC,The Girl in the Red Coat: A Memoir,A2UKKZEITHXC8K,1.0,A book of vanity,I have read hundreds of books about those who ...,14
355154,0967353602,Get All The Facts: HIV does not cause AIDS,A2VVPBN82YWC2,5.0,AIDS is rooted in poverty in Africa,I have read this book and I wish that I could ...,12


In [22]:
rating_df.isna().sum()

RatingID          0
BookTitle         0
UserID            0
Review_Score      0
Review_Summary    0
Review_Text       0
Review_Counts     0
dtype: int64

### 2.2 `books` data exploration <a class="anchor" id="_2.2"></a>

In [23]:
books.sample(1)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
149784,What Matters Most: The Power of Living Your Va...,"The author explores existential angst, dissati...",['Hyrum W. Smith'],http://books.google.com/books/content?id=NJNWe...,http://books.google.com/books?id=NJNWetlo-okC&...,Simon and Schuster,2001-10-09,http://books.google.com/books?id=NJNWetlo-okC&...,['Business & Economics'],1.0


In [24]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [25]:
# books shape
print(f"Total rows: {books.shape[0]}")
print(f"Total columns: {books.shape[1]}")
print(f"Null values: {books.isna().sum().sum()}")

Total rows: 212404
Total columns: 10
Null values: 504645


In [26]:
books.isna().sum()

Title                 1
description       68442
authors           31413
image             52075
previewLink       23836
publisher         75886
publishedDate     25305
infoLink          23836
categories        41199
ratingsCount     162652
dtype: int64

#### 2.2.1 Cleaning `books` data frame: <a class="anchor" id="_2.2.1"></a>

##### A. Duplicate rows

In [27]:
# Calculate the number of duplicated rows
books.duplicated().sum()

0

##### B. Droping columns that are not needed for the analysis
- The columns: `image`, `previewLink`, `infoLink `, will not be used in the analysis



In [28]:
books = books.drop(["image","previewLink","infoLink"],axis=1)

##### C. Droping NaN Values

In [29]:
books.isna().sum()

Title                 1
description       68442
authors           31413
publisher         75886
publishedDate     25305
categories        41199
ratingsCount     162652
dtype: int64

In [30]:
# calculate the percentage of nan values in each column
nan_percent = books.isna().sum() / len(books) * 100
nan_percent

Title             0.000471
description      32.222557
authors          14.789270
publisher        35.727199
publishedDate    11.913617
categories       19.396527
ratingsCount     76.576712
dtype: float64

- Since the `ratingsCount` column has 76% of NaN values, the column will be droped, and this feature will be calculated using the ratings dataframe
- Since the `publisher` column has over 35% of their values NaN, this column will be droped
- Since the `Title` column has only 1 NaN value, this one will be droped
- Since the `description` column has over 32% of NaN values, this records will be droped



In [31]:
# Drop ratingsCount and publisher columns
books = books.drop(["ratingsCount","publisher"],axis=1)

In [32]:
# drop the Title nan values
books.dropna(subset=["Title"],inplace = True)

# drop the description nan values
books.dropna(subset=["description"],inplace = True)

In [33]:
books.shape

(143962, 5)

In [34]:
# Checking the nan values in the books data frame
nan_percent = books.isna().sum() / len(books) * 100
nan_percent

Title            0.000000
description      0.000000
authors          1.533043
publishedDate    0.288965
categories       4.219169
dtype: float64

In [35]:
# Dropping the remaining nan values as they are less than 5% of the data frame

# drop the nan values from the authors column
books.dropna(subset=["authors"],inplace = True)

# drop the nan values from the categories column
books.dropna(subset=["categories"],inplace = True)

# drop the nan values from the publishedDate column
books.dropna(subset=["publishedDate"],inplace = True)

##### D. Cleaning the Title column

In [36]:
# cleaning the Title column

# remove the extra spaces
books["Title"] = books["Title"].str.strip()

# remove the punctuation from the Title column
books['Title'] = books['Title'].str.replace('[^\w\s]','')

##### E. Cleaning the categories column

In [37]:
# Define a function to remove the square brackets 
def remove_square_brackets(text):

    cleaned_text = re.sub(r"^\['|\']$", '', text)
    return cleaned_text

In [38]:
# Apply the function to the "categories" column
books["categories"] = books["categories"].apply(remove_square_brackets)
books.head(5)

,Title,description,authors,publishedDate,categories
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005-01-01,Biography & Autobiography
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,Religion
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005-02,Fiction
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],1996,Religion
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,['Mary Fabyan Windeatt'],2009-01-01,Biography & Autobiography


##### F. Cleaning the authors column

In [39]:
# Apply the function to the "authors" column
books["authors"] = books["authors"].apply(remove_square_brackets)
books.head(10)

,Title,description,authors,publishedDate,categories
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,2005-01-01,Biography & Autobiography
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,David R. Ray,2000,Religion
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,Veronica Haddon,2005-02,Fiction
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,Everett Ferguson,1996,Religion
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,Mary Fabyan Windeatt,2009-01-01,Biography & Autobiography
9,Rising Sons and Daughters: Life Among Japan's ...,Wardell recalls his experience as a foreign st...,Steven Wardell,1995,Social Science
10,Muslim Women's Choices: Religious Belief and S...,Counters the Western views and stereotypes of ...,"Camillia Fawzi El-Solh', 'Judy Mabro",1994-02-17,Religion
11,Dramatica for Screenwriters,Dramatica for Screenwriters by Armando Saldana...,Armando Salda A-Mora,2005-07,Reference
12,Mensa Number Puzzles (Mensa Word Games for Kids),Acclaimed teacher and puzzler Evelyn B. Christ...,Evelyn B. Christensen,2018-11-06,Juvenile Nonfiction
13,Vector Quantization and Signal Compression (Th...,"Herb Caen, a popular columnist for the San Fra...","Allen Gersho', 'Robert M. Gray",2012-12-06,Technology & Engineering


In [40]:
# replacing "', '" with ", " in the authors column
books["authors"] = books["authors"].str.replace("', '",", ")
books.head(10)

,Title,description,authors,publishedDate,categories
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,2005-01-01,Biography & Autobiography
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,David R. Ray,2000,Religion
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,Veronica Haddon,2005-02,Fiction
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,Everett Ferguson,1996,Religion
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,Mary Fabyan Windeatt,2009-01-01,Biography & Autobiography
9,Rising Sons and Daughters: Life Among Japan's ...,Wardell recalls his experience as a foreign st...,Steven Wardell,1995,Social Science
10,Muslim Women's Choices: Religious Belief and S...,Counters the Western views and stereotypes of ...,"Camillia Fawzi El-Solh, Judy Mabro",1994-02-17,Religion
11,Dramatica for Screenwriters,Dramatica for Screenwriters by Armando Saldana...,Armando Salda A-Mora,2005-07,Reference
12,Mensa Number Puzzles (Mensa Word Games for Kids),Acclaimed teacher and puzzler Evelyn B. Christ...,Evelyn B. Christensen,2018-11-06,Juvenile Nonfiction
13,Vector Quantization and Signal Compression (Th...,"Herb Caen, a popular columnist for the San Fra...","Allen Gersho, Robert M. Gray",2012-12-06,Technology & Engineering


#### 2.2.2 `books` Feature engineering: <a class="anchor" id="_2.2.2"></a>

In [41]:
books.sample(3)

,Title,description,authors,publishedDate,categories
98646,Dancing with the Dark: True Encounters with th...,Presents an anthology of real-life supernatura...,Stephen Jones,1999,Fiction
26524,Out of Time's Abyss,This is the tale of Bradley after he left Fort...,Edgar Rice Burroughs,2015-11-04,Fiction
177700,Dick Sutphen Presents Sedona: Psychic Energy V...,"Sedona, Arizona, is considered the location of...","Dick Sutphen, Richard Sutphen",1993,Philosophy


##### A. Creating columns for the 10 first authors from each book

In [42]:
#separate the authors column into multiple columns after ", "
authors = books["authors"].str.split(", ", n = 10, expand = True)
authors.sample(5)

,0,1,2,3,4,5,6,7,8,9,10
28005,Department of Economic and Social Affairs,None,None,None,None,None,None,None,None,None,None
151641,Kenneth C. Davis,None,None,None,None,None,None,None,None,None,None
205732,Lynne Rae Perkins,None,None,None,None,None,None,None,None,None,None
94416,Eithne Loughrey,None,None,None,None,None,None,None,None,None,None
202528,Dick Weissman,None,None,None,None,None,None,None,None,None,None


In [43]:
authors.isna().sum()

0          0
1     106462
2     128149
3     133505
4     134900
5     135364
6     135558
7     135645
8     135709
9     135737
10    135755
dtype: int64

In [44]:
nan_percent = authors.isna().sum() / len(authors) * 100
nan_percent

0      0.000000
1     78.382895
2     94.350000
3     98.293367
4     99.320439
5     99.662060
6     99.804893
7     99.868947
8     99.916067
9     99.936682
10    99.949935
dtype: float64

After analyzing the nan_percentage for books with multiple authors, it has been determined that the first author listed in the column will be considered as the `main_author`.

In [45]:
# adding the first 10 authors columns to the books data frame
books["main_author"] = authors[0]

In [46]:
books.sample(3)

,Title,description,authors,publishedDate,categories,main_author
208707,Not for Glory (Signet),Despite the fact that General Shimon Bar-El ha...,Joel Rosenberg,1989,Fiction,Joel Rosenberg
107816,Rivers and lakes (Planet earth),Rivers and lakes are storehouses of fresh wate...,Rani Iyer,2011-01-01,Nature,Rani Iyer
155313,Bones Burnt Black,A sabotaged passenger craft tumbles toward the...,Stephen Euin Cobb,2004,Fiction,Stephen Euin Cobb


In [47]:
# drop the authors column
books = books.drop(["authors"],axis=1)

##### B. Selecting Columns for the desired data frame

In [48]:
books_df = books[["Title","description","categories","publishedDate","main_author"]]

books_df = books_df.rename(columns={
    "Title": "BookTitle",
    "description": "Book_Description",
    "categories": "Book_Category",
    "publishedDate": "Published_Date",
    "main_author": "Main_Author"
})

books_df.sample(3)

,BookTitle,Book_Description,Book_Category,Published_Date,Main_Author
22256,The Class Struggle and The CTO,"This collection of over 2,000 pages of CTO bes...",Business & Economics,2006-03-01,Aspatore Books
48602,Noir Anxiety,Among the elements that define the classic fil...,Performing Arts,2003,Kelly Oliver
114202,Ex-Virgin,Life is a journey. But as 23-year-old New Zeal...,Fiction,2009,Rachel Knowles


------

## 3. Merging `books` and `ratings` data frames <a class="anchor" id="_3"></a>

In [49]:
rating_df.sample(1)

,RatingID,BookTitle,UserID,Review_Score,Review_Summary,Review_Text,Review_Counts
1424797,0758207948,The River Devil,AFVQZQ8PW0L,5.0,terrific historical romance,In December 1871 in New York Nicholas Lennox w...,19


In [50]:
books_df.sample(1)

,BookTitle,Book_Description,Book_Category,Published_Date,Main_Author
114281,Lie of the Mind,THE STORY: Involves two desperate families con...,Drama,1986,Sam Shepard


Merge the two data frames on the BookTitle column

In [51]:
book_rating = pd.merge(books_df, rating_df, on='BookTitle', how='inner')
book_rating.sample(3)

,BookTitle,Book_Description,Book_Category,Published_Date,Main_Author,RatingID,UserID,Review_Score,Review_Summary,Review_Text,Review_Counts
1522316,Moby Dick or the White Whale,Moby Dick is a novel by American writer Herman...,Fiction,2022-02-01,Herman Melville,B000NPQOMA,A293JGMF1JND0Y,5.0,"""I'd smite the sun if it insulted me""",Easily one of the greatest novels of all-time....,451
957211,"Nicolae: Left Behind, Volume 3",Continues the story of the Tribulation Force a...,Fiction,2011-03-16,Tim LaHaye,B00017JIU0,A69Z3F9A2ILNP,4.0,What can I say...,I don't know what I can say about his book ser...,237
996764,Assignment in Brittany,He stared at the unfamiliar watch on his wrist...,Fiction,2013-03-05,Helen Macinnes,B000NXGJVS,A1AES697PC2IW5,5.0,Olden Golden Suspence +++,"""Assignment in Brittany"" is a prime exemplar o...",54


In [52]:
book_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796852 entries, 0 to 1796851
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   BookTitle         object 
 1   Book_Description  object 
 2   Book_Category     object 
 3   Published_Date    object 
 4   Main_Author       object 
 5   RatingID          object 
 6   UserID            object 
 7   Review_Score      float64
 8   Review_Summary    object 
 9   Review_Text       object 
 10  Review_Counts     int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 150.8+ MB


In [53]:
book_rating.shape

(1796852, 11)

In [54]:
book_rating.isna().sum()

BookTitle           0
Book_Description    0
Book_Category       0
Published_Date      0
Main_Author         0
RatingID            0
UserID              0
Review_Score        0
Review_Summary      0
Review_Text         0
Review_Counts       0
dtype: int64

### 3.1 Checking for duplicates  <a class="anchor" id="_3.1"></a>

In [55]:
# Drop the duplicated rows
book_rating.drop_duplicates(inplace=True)
book_rating.shape

(1788350, 11)

### 3.2 Feature Engineering  <a class="anchor" id="_3.2"></a>

In [56]:
book_rating.sample(3)

,BookTitle,Book_Description,Book_Category,Published_Date,Main_Author,RatingID,UserID,Review_Score,Review_Summary,Review_Text,Review_Counts
1254855,Time for Bed Gift Set: [Night-light and Board ...,"A little boy gets ready for bed, with his own ...",Juvenile Fiction,2004-07-01,Annette Norris,B0001FZG9Y,A17NSO6EVIIHV,3.0,Time for Bed,"This was an appealing book for a small child, ...",174
974874,Look Alikes Jr.,"Welcome to Look-Alike Land, where the more you...",Juvenile Fiction,2003-10-17,Joan Steiner,0316890731,A3FPY7XBMK487R,5.0,"Two covers, same book!",I ordered both Look Alikes Jr. and Look Alikes...,4
1610045,Let's Talk About S-E-X: A Guide for Kids 9 to ...,Provides an introduction to human sexual devel...,Juvenile Nonfiction,2005,Sam Gitchel,1931863180,A1EGBF3ROFYJAC,4.0,Great book!,This is a great book for disabled teen to adul...,14


In [57]:
# Create a new variable with the average rating per each book title
Average_Rating = book_rating.groupby('BookTitle')['Review_Score'].mean()
Average_Rating.head(5)

BookTitle
" Film technique, " and, " Film acting "                                                           4.500000
" We'll Always Have Paris": The Definitive Guide to Great Lines from the Movies                    5.000000
"... And Poetry is Born ..." Russian Classical Poetry                                              4.000000
"A Truthful Impression of the Country": British and American Travel Writing in China, 1880-1949    4.000000
"A" IS FOR ALIBI                                                                                   3.707182
Name: Review_Score, dtype: float64

In [58]:
# Add the average rating column to the book_rating data frame
book_rating["Average_Rating"] = book_rating["BookTitle"].map(Average_Rating)
book_rating.sample(3)

,BookTitle,Book_Description,Book_Category,Published_Date,Main_Author,RatingID,UserID,Review_Score,Review_Summary,Review_Text,Review_Counts,Average_Rating
472634,Hunting with the bow and arrow,"""Hunting with the Bow & Arrow"" by Saxton T. Po...",Fiction,2019-11-21,Saxton T. Pope,B0007E0M34,AXI9T6QJ7DHFD,5.0,Excellent Look Back,"For every student of Native America, aborigina...",52,4.038462
1305952,Power Misses: Essays Across (Un)Popular Cultur...,David James insists that popular resistance to...,Art,1996-12-17,David E. James,1859848060,A2D534V65LDWJH,5.0,major book about the counter-culture,A major book about the counter-culture from W....,1,5.000000
1411610,The witch of Blackbird Pond (A Dell book),"Now an orphan, Kit Tyler travels from Barbados...",Juvenile Fiction,1958,Elizabeth George Speare,B0006W9YL8,A1DNIU1OBDA779,4.0,Witch Of Blackbird Pond is nice,The Witch of Blackbird Pond by Elizabeth Georg...,234,4.416309


---

In [59]:
print("Merged Data frame shape:", book_rating.shape)
print("Number of Unique Book Titles:",len(book_rating["BookTitle"].unique()))
print("Number of Unique Authors:",len(book_rating["Main_Author"].unique()))
print("Number of Unique Book Categories:",len(book_rating["Book_Category"].unique()))
print("Number of Unique UserID:",len(book_rating["UserID"].unique()))

Merged Data frame shape: (1788350, 12)
Number of Unique Book Titles: 132368
Number of Unique Authors: 88829
Number of Unique Book Categories: 5196
Number of Unique UserID: 821831


## 4. Save Processed Data <a class="anchor" id="_4"></a>

In [60]:
# Pickle the DataFrame
book_rating.to_pickle('/Users/rurickgranados/Desktop/Capstone.nosync/book_rating.pkl')
